In [1]:
!pip install mne
!pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import os

path_to_this_work = '/content/drive/MyDrive/fNIRS_Deeplearning_Test'
os.chdir(path_to_this_work)
path_to_src = path_to_this_work + '/src'

sys.path.insert(0, path_to_src)
os.environ['PYTHONPATH'] += (":"+path_to_src)
import Dataset_training_schema as dts
import train_model as tm

In [4]:
import numpy as np
import math

In [5]:
import wandb
wandb.login(key="550fa288f18a4938b6519ecd67d11309cfd9d51e")

wandb: Currently logged in as: cphnycu (bbccii). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import CSP
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

def runClassifyCSP_LDA(data, label, testData=None, testLabel=None):
    # 初始化 CSP 和 LDA
    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
    lda = LinearDiscriminantAnalysis()

    if testData is None:
      #SD
        predictions = []
        true_labels = []

        loo = LeaveOneOut()

        for train_index, test_index in loo.split(data):

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = label[train_index], label[test_index]

            # CSP 特徵提取
            X_train_csp = csp.fit_transform(X_train, y_train)
            X_test_csp = csp.transform(X_test)

            # LDA 分類
            lda.fit(X_train_csp, y_train)
            prediction = lda.predict(X_test_csp)

            predictions.append(prediction[0])
            true_labels.append(y_test[0])

        accuracy = accuracy_score(true_labels, predictions)
    else:
      #SI

        # CSP 特徵提取
        X_train_csp = csp.fit_transform(data, label)
        X_test_csp = csp.transform(testData)

        # LDA 分類
        lda.fit(X_train_csp, label)
        prediction = lda.predict(X_test_csp)

        accuracy = accuracy_score(testLabel, prediction)

    return accuracy * 100


In [7]:
def main(dataset,duration,sub,scheme):                          # 單一測試
# def main():                                         # 執行wandb的部分

  ## 要執行wandb時---start
  # wandb.init()
  # # Config is a variable that holds and saves hyperparameters and inputs
  # config = wandb.config
  # dataset = config.dataset
  # duration = config.duration
  # sub = int(config.sub)
  # scheme = config.scheme
  # method = config.method
  ## 要執行wandb時---end


  if (int(sub)>30 and dataset=='preprocess') or (int(sub)>29 and dataset in ['MI','MA']):
    wandb.finish()
    return 0


  path_dataset='./dataset/'+ dataset + '_' + duration + '/'
  s1=path_dataset+'S'
  s2='.mat'
  sub_range = 31 if dataset=='preprocess' else 30

  if scheme=="SI":

    # 檔案讀取
    test, train = dts.leave_subject_out(s1,s2, data_range=range(1,sub_range), test_id=int(sub))
    acc =  runClassifyCSP_LDA(train.HbO, np.argmax(train.labels,axis=1), testData=test.HbO, testLabel=np.argmax(test.labels,axis=1))

    ## 要執行wandb時---start
    # wandb.log({"Accuracy": acc})
    # wandb.finish()
    ## 要執行wandb時---end

    print('ACC: ',acc)
    return 0



  elif scheme=="SD":

    # 檔案讀取
    data = dts.read_one_file(s1,s2,sub)

    acc = runClassifyCSP_LDA(data.HbO,np.argmax(data.labels,axis=1))

    ## 要執行wandb時---start
    # wandb.log({"Accuracy": acc})
    # wandb.finish()
    ## 要執行wandb時---end

    print('ACC: ',acc)
    return 0


In [ ]:
projectName = 'SVM_KNN_CSP-LDA_eval'
sweepID = 'ui4bkq49'
sweep_agent = 'cphnycu/' + projectName + '/' + sweepID
wandb.agent(sweep_agent, main)

In [8]:
dataset = 'preprocess'
duration = '15s'
sub = 15
scheme = 'SD'
main(dataset,duration,sub,scheme)

Computing rank from data with rank=None
    Using tolerance 0.0071 (2.2e-16 eps * 20 dim * 1.6e+12  max singular value)
    Estimated rank (mag): 20
    MAG: rank 20 computed from 20 data channels with 0 projectors
Reducing data rank from 20 -> 20
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0071 (2.2e-16 eps * 20 dim * 1.6e+12  max singular value)
    Estimated rank (mag): 20
    MAG: rank 20 computed from 20 data channels with 0 projectors
Reducing data rank from 20 -> 20
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0052 (2.2e-16 eps * 20 dim * 1.2e+12  max singular value)
    Estimated rank (mag): 20
    MAG: rank 20 computed from 20 data channels with 0 projectors
Reducing data rank from 20 -> 20
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0072 (2.2e-16 eps * 20 dim * 1.6e+12  max singular value)
    

0